In [48]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/ob/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'interactive']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [49]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import datetime

from matplotlib.colors import ListedColormap
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

import pickle
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from tqdm import tqdm

from ipywidgets import widgets
from IPython.display import display

from __future__ import print_function
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets

In [52]:
regions = pd.read_csv('regions.csv',sep = ';')

yellowmap = pd.read_csv('yellowmap.csv',sep = ',',index_col=[0],parse_dates=True)
yellow_avg = regions
yellow_avg['avg_departure'] = regions.region.apply(lambda x : yellowmap[yellowmap.columns[x-1]].mean())
#yellow_avg = yellow_avg[yellow_avg.avg_departure >= 5]

with open('dfres_c7.pickle', 'rb') as f:
    dfres_c = pickle.load(f) 
date_list_june = [datetime.datetime.strptime("2016-05-31 23:00", "%Y-%m-%d %H:%M") + relativedelta(hours=x) 
             for x in range(0,715)]  
preddf = {}
realdf = {}
for d in tqdm(date_list_june[:10]):
    preddf[d] = np.array([ dfres_c[(dfres_c.date == d) & (dfres_c.region == r)].pred.values[0] if  len(dfres_c[(dfres_c.date == d) & (dfres_c.region == r)])>0 else 0 for r in regions.region.values])


100%|██████████| 10/10 [00:33<00:00,  3.30s/it]


In [79]:
import geojson
import json

import folium
from folium import plugins
import folium.colormap as cmap
from IPython.display import display

In [108]:
plt.figure(figsize(15,7))
list_of_features = []
for ind in regions.index:
    poly = geojson.Polygon([[[regions.iloc[ind].west, regions.iloc[ind].south], 
                             [regions.iloc[ind].west, regions.iloc[ind].north],
                             [regions.iloc[ind].east, regions.iloc[ind].north], 
                             [regions.iloc[ind].east, regions.iloc[ind].south],
                             [regions.iloc[ind].west, regions.iloc[ind].south]]])
    feature = geojson.Feature(geometry=poly, id=str(ind), properties={"name": "reg. {}".format(int(regions.iloc[ind].region))})
    list_of_features.append(feature)
features = geojson.FeatureCollection(list_of_features)

data_reg_mean = preddf[date_list_june[0]]
vmin = dfres_c.min()
vmax = dfres_c.max()
width = vmax - vmin
color_func = cmap.LinearColormap(['white', 'green', 'cyan', 'blue', 'magenta', 'red'],
                               vmax = vmax, vmin = vmin,
                               index=[vmin, 20, 0.3*width, 0.6*width, 0.9*width, 1.0*vmax])


map_nyc = folium.Map(location=[(nyc_latitude_south+nyc_latitude_north)/2, (nyc_longitude_east+nyc_longitude_west)/2],
                     zoom_start = 11, tiles= 'cartodbpositron')

gj = folium.GeoJson(
    features,
    style_function=lambda feature: {
        'fillColor': color_func(data_reg_mean[int(feature['id'])]),
        'color': 'black',
        'weight': 0.2,
        'dashArray': '5, 5'
    }
)
gj.add_to(map_nyc)

def f (x):
    print(x)
    #print(preddf[date_list_june[x]])
    data_reg_mean = preddf[x]
    gj.style_function = lambda feature: {
        'fillColor': color_func(data_reg_mean[int(feature['id'])]),
        'color': 'black',
        'weight': 0.2,
        'dashArray': '5, 5'
    }
    display(map_nyc)

color_func.caption = "Mean value of trips"
map_nyc.add_child(color_func)   
interact(f, x=widgets.SelectionSlider(
    options=date_list_june[:10],
    value=date_list_june[0],
    description='June 2016',
    disabled=False,

    continuous_update=False,
    orientation='horizontal',
    readout=True,

));

Отобра

In [137]:
def f1(x):
    df1 = pd.DataFrame(dfres_c[dfres_c.region == x].values,columns=['region', 'date', 'quantity','lr_pred','pred'],index= dfres_c[dfres_c.region == 1075].date.values)
    df1.quantity.plot()
    df1.pred.plot(color='r')
    plt.ylabel(u'Количество поездок')
    plt.legend([u'Факт',u'Предсказание'])
    plt.show()
plt.figure(figsize(15,7))
    
interact(f1, x= widgets.Dropdown(
    options=np.unique(dfres_c.region.values).tolist(),
    value=1075,
    description=u'Номер региона:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
    ))

<function __main__.f1>

In [ ]:
widgets.HTML(
    value="Hello <b>World</b>",
    placeholder='Some HTML',
    description='Some HTML',
    disabled=False
)